# Figure 5: Peak Picking

In [1]:
import pandas as pd 
from bokeh.io import output_notebook, export_svg
from bokeh.plotting import show

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
output_notebook()

Loading BokehJS ...

#### Load Data

In [4]:
from massdash.loaders import SqMassLoader

loader = SqMassLoader(dataFiles="../../test/test_data/xics/test_chrom_1.sqMass", 
                      rsltsFile="../../test/test_data/osw/test_data.osw" )

transitionGroup = list(loader.loadTransitionGroups("NKESPT(UniMod:21)KAIVR(UniMod:267)", 3).values())[0]

#### Initiate Peak Pickers

In [5]:
from massdash.peakPickers import MRMTransitionGroupPicker, ConformerPeakPicker
from massdash.loaders import SpectralLibraryLoader

pickerMRM_Default = MRMTransitionGroupPicker('sgolay', sgolay_frame_length=11, sgolay_polynomial_order=3)
pickerMRM_Wide = MRMTransitionGroupPicker('sgolay', sgolay_frame_length=13, sgolay_polynomial_order=3)
pickerConformer = ConformerPeakPicker(SpectralLibraryLoader("../../test/test_data/osw/test_data.osw"), "../../massdash/assets/models/conformer/base_cape.onnx", prediction_type='sigmoided')

torch is not installed. Please install it using 'pip install torch'.
torchmetrics is not installed. Please install it using 'pip install torchmetrics'.


#### Perform Peak Picking

In [6]:
topFeatureMRM_Default = pickerMRM_Default.pick(transitionGroup)[0]
topFeatureMRM_Wide = pickerMRM_Wide.pick(transitionGroup)[0]
topFeatureConformer = pickerConformer.pick(transitionGroup)[0]

### for conformer adjust intensity
topFeatureConformer.areaIntensity = 2e5

[2024-03-20 08:47:17,008] ConformerPeakPicker - INFO - Loading model...
[2024-03-20 08:47:17,838] ConformerPeakPicker - INFO - Preprocessing data...
[2024-03-20 08:47:17,844] ConformerPeakPicker - INFO - Predicting...
[2024-03-20 08:47:18,038] ConformerPeakPicker - INFO - Getting predicted boundaries...
[2024-03-20 08:47:18,040] ConformerPeakPicker - INFO - Peak info: {'precursor': [{'max_idx': 88, 'start_idx': 83, 'end_idx': 93, 'score': array([1.5621653], dtype=float32), 'rt_apex': 865.7, 'rt_start': 847.6, 'rt_end': 883.9, 'int_apex': 15790.191329174686}]}


#### Plot

In [7]:
from massdash.plotting import InteractivePlotter, PlotConfig

plotConfig = PlotConfig()
plotConfig.smoothing_dict = dict(type='none')
plotConfig.include_ms1 = False

plotter = InteractivePlotter(plotConfig)

fig = plotter.plot(transitionGroup, 
                   features=[topFeatureMRM_Default, topFeatureMRM_Wide, topFeatureConformer], 
                   feature_legend_labels=['PeakPickerMRM sgolay_frame_length=11','PeakPickerMRM sgolay_frame_legnth=13', 'Conformer: Sigmoided'])



fig.y_range.end = 2e4
fig.x_range.start = 700
fig.x_range.end = 1000
fig.axis.axis_label_text_font_size= '16pt'
fig.axis.major_label_text_font_size = '16pt'
fig.legend.label_text_font_size = '14pt'
fig.legend.title_text_font_size = '14pt'
fig.toolbar_location = None

show(fig)

fig.output_backend = 'svg'

export_svg(fig, filename='Figure_Peak_Picking.svg')

['Figure_Peak_Picking.svg']